In [6]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'mejiro':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [7]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from glob import glob
import multiprocessing
import pandas as pd
from tqdm import tqdm
import time
from copy import deepcopy
from multiprocessing import Pool
from hydra import initialize, compose
import pickle
import pandas as pd

from mejiro.plots import diagnostic_plot, plot
from mejiro.utils import util
from mejiro.helpers import pyhalo

In [8]:
def add(tuple):
    from mejiro.helpers import pyhalo
    from mejiro.utils import util

    # unpack tuple
    (lens, pipeline_params, output_dir) = tuple

    # unpack pipeline_params
    subhalo_cone = pipeline_params['subhalo_cone']
    los_normalization = pipeline_params['los_normalization']

    # add CDM subhalos
    try:
        # use same CDM subhalos
        # lens.add_subhalos(*pyhalo.unpickle_subhalos('/data/bwedig/roman-pandeia/output/pickles/pyhalo/cdm_subhalos_tuple'))

        # TODO update with subhalo_cone param from hydra

        # randomly generate CDM subhalos
        lens.add_subhalos(*pyhalo.generate_CDM_halos(lens.z_lens, lens.z_source, cone_opening_angle_arcsec=subhalo_cone, LOS_normalization=los_normalization))
        
        # pickle
        pickle_target = os.path.join(output_dir, f'lens_with_subhalos_{lens.uid}')
        util.pickle(pickle_target, lens)
        return 1
    except:
        # print(traceback.format_exc())
        return None

In [9]:
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, repo_dir, pickle_dir = config.machine.array_dir, config.machine.data_dir, config.machine.repo_dir, config.machine.pickle_dir
util.create_directory_if_not_exists(pickle_dir)

In [10]:
lens_dict = {}
for band in util.hydra_to_dict(config.pipeline)['band']:
    # open pickled lens list
    lens_list = util.unpickle(os.path.join(pickle_dir, f'01_skypy_output_lens_list_{band.lower()}'))
    lens_dict[band] = lens_list

In [11]:
df = pd.DataFrame(lens_dict)

In [12]:
df

,F106,F129,F158,F184
0,<mejiro.lenses.lens.Lens object at 0x7f3d3ddce...,<mejiro.lenses.lens.Lens object at 0x7f3d3ddcd...,<mejiro.lenses.lens.Lens object at 0x7f3d3dd30...,<mejiro.lenses.lens.Lens object at 0x7f3d3ddcf...
1,<mejiro.lenses.lens.Lens object at 0x7f3d2a87a...,<mejiro.lenses.lens.Lens object at 0x7f3d2a87b...,<mejiro.lenses.lens.Lens object at 0x7f3d2a3b0...,<mejiro.lenses.lens.Lens object at 0x7f3d2a3f1...
2,<mejiro.lenses.lens.Lens object at 0x7f3d2a87b...,<mejiro.lenses.lens.Lens object at 0x7f3d2a3b0...,<mejiro.lenses.lens.Lens object at 0x7f3d2a3f0...,<mejiro.lenses.lens.Lens object at 0x7f3d2a45d...
3,<mejiro.lenses.lens.Lens object at 0x7f3d2a3b0...,<mejiro.lenses.lens.Lens object at 0x7f3d2a3b2...,<mejiro.lenses.lens.Lens object at 0x7f3d2a424...,<mejiro.lenses.lens.Lens object at 0x7f3d2a2c9...
4,<mejiro.lenses.lens.Lens object at 0x7f3d2a3b1...,<mejiro.lenses.lens.Lens object at 0x7f3d2a3b3...,<mejiro.lenses.lens.Lens object at 0x7f3d2a427...,<mejiro.lenses.lens.Lens object at 0x7f3d2a338...
...,...,...,...,...
2821,<mejiro.lenses.lens.Lens object at 0x7f3d20755...,<mejiro.lenses.lens.Lens object at 0x7f3d1ba40...,<mejiro.lenses.lens.Lens object at 0x7f3d15d36...,<mejiro.lenses.lens.Lens object at 0x7f3d10060...
2822,<mejiro.lenses.lens.Lens object at 0x7f3d20756...,<mejiro.lenses.lens.Lens object at 0x7f3d1ba40...,<mejiro.lenses.lens.Lens object at 0x7f3d15d37...,<mejiro.lenses.lens.Lens object at 0x7f3d10061...
2823,<mejiro.lenses.lens.Lens object at 0x7f3d20757...,<mejiro.lenses.lens.Lens object at 0x7f3d1ba41...,<mejiro.lenses.lens.Lens object at 0x7f3d15d6c...,<mejiro.lenses.lens.Lens object at 0x7f3d10062...
2824,<mejiro.lenses.lens.Lens object at 0x7f3d2058c...,<mejiro.lenses.lens.Lens object at 0x7f3d1ba42...,<mejiro.lenses.lens.Lens object at 0x7f3d15d6c...,<mejiro.lenses.lens.Lens object at 0x7f3d10063...
